Data Loading and Preprocessing

This notebook handles loading and cleaning the datasets, merging them into a unified dataframe, and preparing the data for analysis. The goal is to structure the data to answer key questions about ferritin deficiency in women, such as:

Which demographic variables are most strongly associated with ferritin deficiency?
What body measurements show significant correlation with ferritin deficiency?
How complete and reliable is the data for these variables?
Are there any obvious data quality issues that need to be addressed before further analysis?

Installing Libraries

In [89]:
import pandas as pd
from sklearn.impute import KNNImputer

Bring In Data

In [90]:
# defining file paths
bmx_file_path = '/Users/abbeylasater/Desktop/ferritin-risk-model/BMX_L.xpt.xpt'
demo_file_path = '/Users/abbeylasater/Desktop/ferritin-risk-model/DEMO_L.xpt.xpt'
fertin_file_path = '/Users/abbeylasater/Desktop/ferritin-risk-model/FERTIN_L.xpt.xpt'

# Read the XPT file
bmx_df = pd.read_sas(bmx_file_path, format='xport')
demo_df = pd.read_sas(demo_file_path, format='xport')
fertin_df = pd.read_sas(fertin_file_path, format='xport')



In [91]:
# display top of each
print(bmx_df.head())
print(demo_df.head())
print(fertin_df.head())

       SEQN  BMDSTATS  BMXWT  BMIWT  BMXRECUM  BMIRECUM  BMXHEAD  BMIHEAD  \
0  130378.0       1.0   86.9    NaN       NaN       NaN      NaN      NaN   
1  130379.0       1.0  101.8    NaN       NaN       NaN      NaN      NaN   
2  130380.0       1.0   69.4    NaN       NaN       NaN      NaN      NaN   
3  130381.0       1.0   34.3    NaN       NaN       NaN      NaN      NaN   
4  130382.0       3.0   13.6    NaN       NaN       1.0      NaN      NaN   

   BMXHT  BMIHT  ...  BMXLEG  BMILEG  BMXARML  BMIARML  BMXARMC  BMIARMC  \
0  179.5    NaN  ...    42.8     NaN     42.0      NaN     35.7      NaN   
1  174.2    NaN  ...    38.5     NaN     38.7      NaN     33.7      NaN   
2  152.9    NaN  ...    38.5     NaN     35.5      NaN     36.3      NaN   
3  120.1    NaN  ...     NaN     NaN     25.4      NaN     23.4      NaN   
4    NaN    1.0  ...     NaN     NaN      NaN      1.0      NaN      1.0   

   BMXWAIST  BMIWAIST  BMXHIP  BMIHIP  
0      98.3       NaN   102.9     NaN  


pre-processing steps:
Handle duplicates, fix inconsistent column names.
Convert data types as needed (e.g., dates, categorical variables).

Check for missing values — where and how much?
Identify any obviously corrupted or outlier data.
Look for inconsistent values or categories.

Rename columns for clarity.
Create meaningful data types (category, datetime, etc.).
If needed, filter down to your target population (e.g., women aged 18-50).

In [92]:
# joining datasets
df = pd.merge(fertin_df, bmx_df, on='SEQN', how='left')
df = pd.merge(df, demo_df, on='SEQN', how='left')

In [93]:
# dropping any rows with no ferritin level
df = df[~df['LBXFER'].isnull()]

In [94]:
# Selecting necessary columns
df = df[['SEQN',
         'SDDSRVYR',
         'RIDSTATR',
         'RIAGENDR',
         'RIDAGEYR',
         'RIDAGEMN',
         'RIDRETH1',
         'RIDRETH3',
         'RIDEXMON',
         'RIDEXAGM',
         'BMXWT',
         'BMXHT',
         'BMXBMI',
         'BMXWAIST',
         'LBXFER']]


#  columns meaning
        BMXWT : weight (kg)
         'RIAGENDR', gender
         'RIDAGEYR' : age (years)
         'RIDAGEMN' : age (months)
         'RIDRETH1' : race-ethnicity cat
         'RIDRETH3': race-ethnicity cat
         'RIDEXMON', month survery was perf
         'RIDEXAGM', age in mo. at time of exam
         'BMXWT' : weight (kg)
         'BMXHT': standing height (cm)
         'BMXBMI' : body mass index (kg/m)
         'BMXWAIST': waist_circum
         'LBXFER': ferritin level (ng/ml)

In [95]:
# Threshold for dropping nulls (60%)
threshold = 0.6

# Calculate the fraction of nulls in each column
null_fraction = df.isnull().mean()

# Filter columns with > threshold
cols_to_drop = null_fraction[null_fraction > threshold].index

# Drop those columns
df = df.drop(columns=cols_to_drop)

In [96]:
# removing outliers using IQR
def remove_outliers(df, columns):
    for col in columns:
        if df[col].dtype != 'object':
            q1 = df[col].quantile(.25)
            q3 = df[col].quantile(.75)
            iqr = q3 - q1
            lower = q1 - 1.5 * iqr
            upper = q3 + 1.5 * iqr
            df = df[(df[col] >= lower) & (df[col]<= upper)]
    return df

# columns to remove outliers from
outliers_cols = []

df = remove_outliers(df, outliers_cols)

In [97]:
print(df.head())

       SEQN  SDDSRVYR  RIDSTATR  RIAGENDR  RIDAGEYR  RIDRETH1  RIDRETH3  \
0  130380.0      12.0       2.0       2.0      44.0       2.0       2.0   
2  130382.0      12.0       2.0       1.0       2.0       3.0       3.0   
4  130390.0      12.0       2.0       2.0      31.0       3.0       3.0   
5  130391.0      12.0       2.0       2.0      33.0       3.0       3.0   
6  130395.0      12.0       2.0       2.0      33.0       5.0       7.0   

   RIDEXMON  BMXWT  BMXHT  BMXBMI  BMXWAIST  LBXFER  
0       1.0   69.4  152.9    29.7      93.5    13.3  
2       2.0   13.6    NaN     NaN       NaN    24.4  
4       2.0  122.7  163.3    46.0     131.0    52.1  
5       2.0  116.3  172.8    38.9     123.8    91.0  
6       2.0  138.4  159.2    54.6     142.5   113.0  


In [98]:
# filling null values using scikit-learn
features = [
    'RIAGENDR',  # gender
    'RIDAGEYR',  # age
    'RIDRETH1',  # race/ethnicity
    'RIDEXMON',  # month of exam
    'BMXWT',     # weight
    'BMXHT',     # height
    'BMXBMI',    # bmi
    'BMXWAIST'   # waist 
]

# taking subset of og dataframe for imputation
df_to_impute = df[features]

# initializing imputer
imputer = KNNImputer(n_neighbors = 5)

# perform imputation
imputed_array = imputer.fit_transform(df_to_impute)

# replacing w imputed cols
df[features] = imputed_array

In [99]:
print(df.head())

       SEQN  SDDSRVYR  RIDSTATR  RIAGENDR  RIDAGEYR  RIDRETH1  RIDRETH3  \
0  130380.0      12.0       2.0       2.0      44.0       2.0       2.0   
2  130382.0      12.0       2.0       1.0       2.0       3.0       3.0   
4  130390.0      12.0       2.0       2.0      31.0       3.0       3.0   
5  130391.0      12.0       2.0       2.0      33.0       3.0       3.0   
6  130395.0      12.0       2.0       2.0      33.0       5.0       7.0   

   RIDEXMON  BMXWT   BMXHT  BMXBMI  BMXWAIST  LBXFER  
0       1.0   69.4  152.90   29.70      93.5    13.3  
2       2.0   13.6   91.02   16.32      49.0    24.4  
4       2.0  122.7  163.30   46.00     131.0    52.1  
5       2.0  116.3  172.80   38.90     123.8    91.0  
6       2.0  138.4  159.20   54.60     142.5   113.0  


Writing Dataframe

In [100]:
df.to_csv('cleaned_data.csv', index=False)
